In [50]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from natsort import natsorted
import numpy as np
import matplotlib.animation as animation
import xarray as xr
import h5py
import imageio
import matplotlib
import gc
import sys
import io

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(rf'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

tidy3dAPI = os.environ["API_TIDY3D_KEY"]
plt.rc('font', family='Arial')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
# output_file = rf"G:\20250423 2D SHU Statistics\data\20250423 Beam Spreading Time L=14\intensities_exit_average.h5"
# output_file = rf"G:\20250422 2D SHU Statistics\data\20250422 Beam Spreading Time L=14\intensities_exit_average.h5"
output_file = rf"G:\20250428 2D SHU Statistics\data\20250428 Beam Spreading Time L=14 Absorbers\intensities_exit_average.h5"

def print_h5_keys(h5_group, indent=0):
    for key in h5_group.keys():
        print(' ' * indent + key)
        # If the key is a group, recurse into it
        if isinstance(h5_group[key], h5py.Group):
            print_h5_keys(h5_group[key], indent + 4)

# Open the HDF5 file in read mode and print all keys
with h5py.File(output_file, 'r') as h5f:
    print_h5_keys(h5f)

0_ref
    nu_0.14-0.17
    nu_0.14-0.17_flux
    nu_0.14-0.17_flux_time
    nu_0.14-0.17_t
    nu_0.14-0.17_x
    nu_0.18-0.21
    nu_0.18-0.21_flux
    nu_0.18-0.21_flux_time
    nu_0.18-0.21_t
    nu_0.18-0.21_x
    nu_0.25-0.28
    nu_0.25-0.28_flux
    nu_0.25-0.28_flux_time
    nu_0.25-0.28_t
    nu_0.25-0.28_x
    nu_0.28-0.31
    nu_0.28-0.31_flux
    nu_0.28-0.31_flux_time
    nu_0.28-0.31_t
    nu_0.28-0.31_x
chi_0.31_N_10000_posics
    nu_0.14-0.17
    nu_0.14-0.17_flux
    nu_0.14-0.17_flux_time
    nu_0.14-0.17_t
    nu_0.14-0.17_x
    nu_0.18-0.21
    nu_0.18-0.21_flux
    nu_0.18-0.21_flux_time
    nu_0.18-0.21_t
    nu_0.18-0.21_x
    nu_0.25-0.28
    nu_0.25-0.28_flux
    nu_0.25-0.28_flux_time
    nu_0.25-0.28_t
    nu_0.25-0.28_x
    nu_0.28-0.31
    nu_0.28-0.31_flux
    nu_0.28-0.31_flux_time
    nu_0.28-0.31_t
    nu_0.28-0.31_x
chi_0.36_N_10000_posics
    nu_0.14-0.17
    nu_0.14-0.17_flux
    nu_0.14-0.17_flux_time
    nu_0.14-0.17_t
    nu_0.14-0.17_x
    nu_0.1

In [52]:
data = []
with h5py.File(output_file, 'r') as h5f:
    # Iterate through each group (corresponding to the outer folders)

    for group_name in h5f:
        group = h5f[group_name]
        # Iterate through each dataset (corresponding to the frequencies)
        data_item = []
        i=0
        for dataset_name in group:
            if i%5==0:
                # Read the data from the dataset
                intensity_data = group[dataset_name][:]
                x = group[rf"{dataset_name}_x"][:]
                t = group[rf"{dataset_name}_t"][:]*1e12
                flux = group[rf"{dataset_name}_flux"][:]
                flux_time = group[rf"{dataset_name}_flux_time"][:]*1e12
                data_item.append({"set":dataset_name,"Intensity Data": intensity_data,"x":x,"t":t,"flux":flux,"flux_time":flux_time})
            i+=1
        data.append({"chi":group_name,"data":data_item})


In [53]:
import scipy.integrate
def calculate_IPR(intensity_array,x1):
    values = intensity_array
    integral_1 = scipy.integrate.trapezoid(values**2, x=x1, axis=0)
    integral_2 = scipy.integrate.trapezoid(values, x=x1,axis=0)**2

    return integral_2/integral_1

def moving_average(x, w=3):
    return np.convolve(x, np.ones(w), 'valid') / w

In [54]:
def calculate_sigma(intensity_array, x1):
    x1 = np.asarray(x1)
    values = intensity_array

    # Reshape x1 to broadcast over columns (axis=1)
    x1 = x1[:, np.newaxis]  

    norm = scipy.integrate.trapezoid(values, x=x1[:, 0], axis=0)
    mean = scipy.integrate.trapezoid(values * x1, x=x1[:, 0], axis=0) / norm
    mean_sq = scipy.integrate.trapezoid(values * x1**2, x=x1[:, 0], axis=0) / norm
    sigma = np.sqrt(mean_sq - mean**2)
    return sigma

In [55]:
# from scipy.optimize import curve_fit
# def test(t, b,c):
#     return b * t**c
chi_values = ["Reference", "0.31","0.36","0.37"]
# chi_values = ["Reference", "0.30","0.31","0.32","0.33","0.34","0.35","0.36","0.37","0.38","0.39","0.40"]
# for i, chi in enumerate(data):
#  plt.figure(figsize=(15, 10))
#  plt.plot(np.linspace(0,1,100),40*np.sqrt(np.linspace(0,1,100)),"--",color="red", label=r"$\propto t^{0.5}$")
#  for j, freq in enumerate(chi["data"]):
#     ipr = calculate_IPR(freq["Intensity Data"],x1=freq["x"])
#     plt.scatter(freq["t"],ipr, label=freq["set"])
   
#  plt.legend()
# #  plt.ylim(0,50)
#  plt.title(chi_values[i])
#  plt.ylabel(rf"Output Intensity Length [a]",fontdict={"fontsize":16})
#  plt.xlabel(rf"Time [ps]",fontdict={"fontsize":16})
#  plt.show()
    



In [56]:
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Create a custom colormap
colors = [
              (1, 1, 1),  # White
              (0, 0, 1),  # Blue
              (1, 1, 0),  # Yellow
              (0, 1, 0),  # Green
              (1, 0, 0), #Red
              (0, 0, 0), # Black
              ]  
n_bins = 500  # Number of bins for smooth transition

cmap = mcolors.LinearSegmentedColormap.from_list("custom_colormap", colors, N=n_bins)
freq_values = [rf"$\nu =$0.14 - 0.17", rf"$\nu =$0.18 - 0.21", rf"$\nu =$0.25 - 0.28", rf"$\nu =$0.28 - 0.31"]
normalization_time=[]
for i, chi in enumerate(data):
    
    fig, ax = plt.subplots(ncols=2,nrows=3,figsize=(20,10))
    for j, freq in enumerate(chi["data"]):

        if i == 0:
            index_max_flux_ref =np.argmax(freq["flux"])
            normalization_time.append(freq["flux_time"][index_max_flux_ref])
            
        
        time_mask = (freq["t"]>=0)
        time = freq["t"][time_mask]/normalization_time[j]
        x = freq["x"]
        
        intensity_values  = freq["Intensity Data"][:,time_mask]
        intensity_values[intensity_values/np.max(intensity_values)<=1e-9]=0
        max_values_time_step = (np.max(intensity_values,axis=0))
        intensity_normalized = intensity_values/max_values_time_step
        intensity_normalized[np.isnan(intensity_normalized)] = 0

        # ipr = calculate_IPR(intensity_normalized,x1=x)
        ipr = 2*calculate_sigma(intensity_normalized,x1=x)

        if j == 0: # Ensure it fits the diffusive case 
            ax[0,0].plot(time,45*((time**(1/2))),"--",color="red", label=r"$\propto \sqrt{\tau}$")

        if j in [0,1]:
            ax[0,0].scatter(time[time<=3],ipr[time<=3], label=freq_values[j])
        else:
            ax[0,0].scatter(time[:],(ipr), label=freq_values[j])
        
        mask_intensity = np.where(time>=0)[0]
        if j==0:
            mask_intensity=np.where(np.bool_(time>=0.5) & np.bool_(time<=1.8))[0]
        elif j ==1:
            mask_intensity=np.where(np.bool_(time>=0.5) & np.bool_(time<=2))[0]

        im = ax[(j+2)//2, (j+2)%2].imshow(intensity_normalized[:,mask_intensity], vmin = 0, vmax = 1, 
                 extent =[np.min(time[mask_intensity]),np.max(time[mask_intensity]),np.min(x),np.max(x)], 
                     origin ='lower', cmap=cmap,aspect='auto', interpolation="gaussian") 
           
        ax[(j+2)//2, (j+2)%2].set_title(freq_values[j],fontsize=18)
        ax[(j+2)//2, (j+2)%2].tick_params(which='major', labelsize=16)
        ax[(j+2)//2, (j+2)%2].set_xlabel(r'$\tau$', fontsize=20)
        subplot_label = chr(67 + j)  # chr(65) = 'A', chr(66) = 'B', etc.
        ax[(j+2)//2, (j+2)%2].text(0.01, 0.95, subplot_label, transform=ax[(j+2)//2, (j+2)%2].transAxes,fontsize=20, fontweight='bold', va='top', ha='left')
       
        

        index_max_flux =np.argmax(freq["flux"])
        ax[0,1].plot(freq["flux_time"][:]/normalization_time[j],freq["flux"][:]/freq["flux"][index_max_flux],"o",label=rf"{freq_values[j]}")
        ax[0,1].set_yscale("log")
        ax[0,1].set_title("Normalized Flux in Time",fontsize=18)


    ax[0,0].text(0.01, 0.95, "A", transform=ax[0,0].transAxes, fontsize=20, fontweight='bold', va='top', ha='left')
    ax[0,1].text(0.01, 0.95, "B", transform=ax[0,1].transAxes, fontsize=20, fontweight='bold', va='top', ha='left')

    ax[0,0].set_ylim(0,85)
    ax[0,0].legend(fontsize=18)
    ax[0,1].legend(fontsize=18)
    ax[0,1].set_ylim(1e-12,1)
    fig.text(0.0, 0.5, 'Output Intensity Length (a)', va='center', rotation='vertical', fontsize=18)
    ax[0,1].set_xlabel(r'$\tau$', fontsize=20)
    ax[0,0].set_xlabel(r'$\tau$', fontsize=20)
    ax[0,0].tick_params(which='major', labelsize=16)
    ax[0,1].tick_params(which='major', labelsize=16)

    cbar_ax = fig.add_axes([0.95, 0.10, 0.02, 0.7])  # [left, bottom, width, height]
    cbar = fig.colorbar(im, ax=ax,cax=cbar_ax)
    cbar.ax.tick_params(labelsize=18)
    cbar.set_label(rf"$|E|^2$ Normalized", fontsize=18)
    cbar.ax.yaxis.set_label_position('left')
    fig.suptitle(rf"$\chi=${chi_values[i]}", fontsize=20, x=0.45)
    fig.tight_layout(rect=[0, 0, 0.9, 1])  # Adjust the rect parameter to leave space for the colorbar

    Path(rf"G:\20250428 2D SHU Statistics\data\20250428 Beam Spreading Time L=14 Absorbers\img").mkdir(parents=True,exist_ok=True)
    if i<3:
            plt.close()
            continue
    plt.savefig(rf"G:\20250428 2D SHU Statistics\data\20250428 Beam Spreading Time L=14 Absorbers\img\chi_{chi_values[i]}.pdf")
    plt.close()





C:\Users\HernandF\AppData\Local\Temp\ipykernel_25868\2580530736.py:35: RuntimeWarning: invalid value encountered in divide
  intensity_normalized = intensity_values/max_values_time_step
C:\Users\HernandF\AppData\Local\Temp\ipykernel_25868\3813351107.py:9: RuntimeWarning: invalid value encountered in divide
  mean = scipy.integrate.trapezoid(values * x1, x=x1[:, 0], axis=0) / norm
C:\Users\HernandF\AppData\Local\Temp\ipykernel_25868\3813351107.py:10: RuntimeWarning: invalid value encountered in divide
  mean_sq = scipy.integrate.trapezoid(values * x1**2, x=x1[:, 0], axis=0) / norm
C:\Users\HernandF\AppData\Local\Temp\ipykernel_25868\2580530736.py:92: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect=[0, 0, 0.9, 1])  # Adjust the rect parameter to leave space for the colorbar
